# Final Project File

Group 4<br>
Project Review Date/Time: 06/25/2021 5:00pm est<br>
Instructor Name: David Elliott

## Business Understanding

Notebook clearly explains the project’s value for helping a specific stakeholder solve a real-world problem.<br>
Introduction explains the real-world problem the project aims to solve<br>
Introduction identifies stakeholders who could use the project and how they would use it<br>
Conclusion summarizes implications of the project for the real-world problem and stakeholders<br>



## Data Preparation

Notebook clearly describes the source and properties of the data to show how useful the data are for solving the problem of interest.<br>
Describe the data sources and explain why the data are suitable for the project<br>
Present the size of the dataset and descriptive statistics for all features used in the analysis<br>
Justify the inclusion of features based on their properties and relevance for the project<br>
Identify any limitations of the data that have implications for the project<br>


In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math
import seaborn as sns

%matplotlib inline

df = pd.read_csv('Data/data-clean.csv')
df.drop('Unnamed: 0',axis=1,inplace=True)

#doing this here and not during data cleaning because this split doesnt survive being saved as .csv
df['genres'] = df['genres'].map(lambda x: x.split(","))
df['director'] = df['director'].map(lambda x: x.split(","))

display(df.head(2)),(display(df.info()))

,movie,production_budget,domestic_gross,worldwide_gross,foreign_gross,profit,year,genres,director
0,Pirates of the Caribbean: On Stranger Tides 2011,410600000,241063875,1045663875,804600000,635063875,2011,"[Action, Adventure, Fantasy]",[Rob Marshall]
1,Dark Phoenix 2019,350000000,42762350,149762350,107000000,-200237650,2019,"[Action, Adventure, Sci-Fi]",[Simon Kinberg]


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1349 entries, 0 to 1348
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   movie              1349 non-null   object
 1   production_budget  1349 non-null   int64 
 2   domestic_gross     1349 non-null   int64 
 3   worldwide_gross    1349 non-null   int64 
 4   foreign_gross      1349 non-null   int64 
 5   profit             1349 non-null   int64 
 6   year               1349 non-null   int64 
 7   genres             1349 non-null   object
 8   director           1349 non-null   object
dtypes: int64(6), object(3)
memory usage: 95.0+ KB


None

(None, None)

In [2]:
df.isna().sum()

movie                0
production_budget    0
domestic_gross       0
worldwide_gross      0
foreign_gross        0
profit               0
year                 0
genres               0
director             0
dtype: int64

## Data Analysis

Notebook promotes three recommendations for choosing films to produce.<br>
Uses three or more findings from data analyses to support recommendations<br>
Explains why the findings support the recommendations<br>
Explains how the recommendations would help the new movie studio succeed<br>

### Question 1: Samantha's Section

In [3]:
df3_new = df

In [4]:
#create a column to show the rate of return based on only domestic_gross for each movie
df3_new['roi_domestic'] = (df3_new['domestic_gross'] - df3_new['production_budget']) / df3_new['production_budget'] * 100
df3_new.head()

,movie,production_budget,domestic_gross,worldwide_gross,foreign_gross,profit,year,genres,director,roi_domestic
0,Pirates of the Caribbean: On Stranger Tides 2011,410600000,241063875,1045663875,804600000,635063875,2011,"[Action, Adventure, Fantasy]",[Rob Marshall],-41.289850
1,Dark Phoenix 2019,350000000,42762350,149762350,107000000,-200237650,2019,"[Action, Adventure, Sci-Fi]",[Simon Kinberg],-87.782186
2,Avengers: Age of Ultron 2015,330600000,459005868,1403013963,944008095,1072413963,2015,"[Action, Adventure, Sci-Fi]",[Joss Whedon],38.840250
3,Avengers: Infinity War 2018,300000000,678815482,2048134200,1369318718,1748134200,2018,"[Action, Adventure, Sci-Fi]","[Anthony Russo, Joe Russo]",126.271827
4,Justice League 2017,300000000,229024295,655945209,426920914,355945209,2017,"[Action, Adventure, Fantasy]",[Zack Snyder],-23.658568


In [5]:
#create a column showing rate of return using qorldwide_gross for each movie
df3_new['roi_worldwide'] = (df3_new['worldwide_gross'] - df3_new['production_budget']) / df3_new['production_budget'] * 100
df3_new.head()

,movie,production_budget,domestic_gross,worldwide_gross,foreign_gross,profit,year,genres,director,roi_domestic,roi_worldwide
0,Pirates of the Caribbean: On Stranger Tides 2011,410600000,241063875,1045663875,804600000,635063875,2011,"[Action, Adventure, Fantasy]",[Rob Marshall],-41.289850,154.667286
1,Dark Phoenix 2019,350000000,42762350,149762350,107000000,-200237650,2019,"[Action, Adventure, Sci-Fi]",[Simon Kinberg],-87.782186,-57.210757
2,Avengers: Age of Ultron 2015,330600000,459005868,1403013963,944008095,1072413963,2015,"[Action, Adventure, Sci-Fi]",[Joss Whedon],38.840250,324.384139
3,Avengers: Infinity War 2018,300000000,678815482,2048134200,1369318718,1748134200,2018,"[Action, Adventure, Sci-Fi]","[Anthony Russo, Joe Russo]",126.271827,582.711400
4,Justice League 2017,300000000,229024295,655945209,426920914,355945209,2017,"[Action, Adventure, Fantasy]",[Zack Snyder],-23.658568,118.648403


In [6]:
#I am sorting my worldwide roi column in descending order
final_df_worldwide = df3_new.sort_values(by=['roi_worldwide'], ascending=False)
final_df_worldwide

,movie,production_budget,domestic_gross,worldwide_gross,foreign_gross,profit,year,genres,director,roi_domestic,roi_worldwide
1328,The Gallows 2015,100000,22764410,41656474,18892064,41556474,2015,"[Horror, Mystery, Thriller]","[Travis Cluff, Chris Lofing]",22664.4100,41556.474000
1250,The Devil Inside 2012,1000000,53262945,101759490,48496545,100759490,2012,[Horror],[William Brent Bell],5226.2945,10075.949000
1149,Paranormal Activity 2 2010,3000000,84752907,177512032,92759125,174512032,2010,[Horror],[Tod Williams],2725.0969,5817.067733
1071,Get Out 2017,5000000,176040665,255367951,79327286,250367951,2017,"[Horror, Mystery, Thriller]",[Jordan Peele],3420.8133,5007.359020
1229,Moonlight 2016,1500000,27854931,65245512,37390581,63745512,2016,[Drama],[Barry Jenkins],1756.9954,4249.700800
...,...,...,...,...,...,...,...,...,...,...,...
1212,The Curse of Downers Grove 2015,2000000,0,0,0,-2000000,2015,"[Drama, Horror, Mystery]",[Derick Martini],-100.0000,-100.000000
1211,True Bromance 2011,2000000,0,0,0,-2000000,2011,[Comedy],[Sebastian Doggart],-100.0000,-100.000000
1210,The Walking Deceased 2015,2000000,0,0,0,-2000000,2015,"[Comedy, Horror]",[Scott Dow],-100.0000,-100.000000
1209,Coffee Town 2013,2000000,0,0,0,-2000000,2013,[Comedy],[Brad Copeland],-100.0000,-100.000000


In [7]:
final_df_worldwide = final_df_worldwide.explode('genres')
final_df_worldwide.head()

,movie,production_budget,domestic_gross,worldwide_gross,foreign_gross,profit,year,genres,director,roi_domestic,roi_worldwide
1328,The Gallows 2015,100000,22764410,41656474,18892064,41556474,2015,Horror,"[Travis Cluff, Chris Lofing]",22664.4100,41556.474000
1328,The Gallows 2015,100000,22764410,41656474,18892064,41556474,2015,Mystery,"[Travis Cluff, Chris Lofing]",22664.4100,41556.474000
1328,The Gallows 2015,100000,22764410,41656474,18892064,41556474,2015,Thriller,"[Travis Cluff, Chris Lofing]",22664.4100,41556.474000
1250,The Devil Inside 2012,1000000,53262945,101759490,48496545,100759490,2012,Horror,[William Brent Bell],5226.2945,10075.949000
1149,Paranormal Activity 2 2010,3000000,84752907,177512032,92759125,174512032,2010,Horror,[Tod Williams],2725.0969,5817.067733


In [8]:
final_df_worldwide.shape

(3462, 11)

In [9]:
#filter my datetime column to only show movies released in the last 10 years (since 2011)
filtered_df = final_df_worldwide.loc[(final_df_worldwide['year'].astype(str) >= '2011-01-01')]

genres_list = list(final_df_worldwide['genres'].unique())
genres_list = [x.lower() for x in genres_list]


for genre in genres_list:
    #Using locals() function to Convert a Python string to a Variable Name
    #https://www.pythonpool.com/python-string-to-variable-name/
    locals()[genre] = filtered_df.loc[filtered_df.genres=='Horror']
genres_list

['horror',
 'mystery',
 'thriller',
 'drama',
 'action',
 'biography',
 'fantasy',
 'romance',
 'comedy',
 'music',
 'crime',
 'adventure',
 'animation',
 'family',
 'documentary',
 'sci-fi',
 'sport',
 'history',
 'western',
 'musical',
 'war']

In [10]:
thriller.describe()

,production_budget,domestic_gross,worldwide_gross,foreign_gross,profit,year,roi_domestic,roi_worldwide
count,1.220000e+02,1.220000e+02,1.220000e+02,1.220000e+02,1.220000e+02,122.000000,122.000000,122.000000
mean,1.734221e+07,2.993848e+07,6.751308e+07,3.757460e+07,5.017086e+07,2014.770492,425.751559,914.331049
std,3.013914e+07,4.490587e+07,1.085126e+08,6.781277e+07,9.227040e+07,2.056182,2139.197587,3918.612225
min,2.500000e+04,0.000000e+00,0.000000e+00,0.000000e+00,-3.292904e+07,2012.000000,-100.000000,-100.000000
25%,2.550000e+06,1.287150e+04,3.149845e+05,1.069415e+05,-1.200000e+06,2013.000000,-99.781930,-77.031065
50%,7.450000e+06,1.550038e+07,3.036425e+07,1.217194e+07,1.507740e+07,2015.000000,-1.040390,177.221476
75%,1.792500e+07,4.373346e+07,8.789942e+07,4.621556e+07,6.972356e+07,2016.000000,288.002955,653.666025
max,1.900000e+08,3.274817e+08,6.974580e+08,3.840870e+08,6.624580e+08,2019.000000,22664.410000,41556.474000


NameError: name 'news' is not defined

In [ ]:
horror.describe()

In [ ]:
mystery.describe()

In [ ]:
comedy.describe()

In [ ]:
romance.describe()

In [ ]:
sport.describe()

In [ ]:
drama.describe()

In [ ]:
action.describe()

In [ ]:
biography.describe()

In [ ]:
fantasy.describe()

In [ ]:
music.describe()

In [ ]:
documentary.describe()

In [ ]:
crime.describe()

In [ ]:
scifi.describe()

In [ ]:
adventure.describe()

In [ ]:
animation.describe()

In [ ]:
family.describe()

In [ ]:
history.describe()

In [ ]:
musical.describe()

In [ ]:
western.describe()

In [ ]:
war.describe()

In [ ]:
filtered_df.reset_index(drop=True)

In [ ]:
fig_dims = (20, 14)
fig, ax = plt.subplots(figsize=fig_dims)
sns.boxplot( y=filtered_df["genres"], x=filtered_df["roi_worldwide"], palette="Blues", width=0.6, showfliers = False);
plt.show()

In [ ]:
#filtered_df.drop(filtered_df[(filtered_df['genres'] =='Crime') | (filtered_df['genres'] == 'Sport')].index, inplace=True)
#filtered_df

#this works, it just seems innefficient to type out 17 genres

#top_5 = ['animation', 'mystery', 'adventure', 'sci-fi', 'comedy']

#for index, row in filtered_df.iterrows():
#    for top in top_5:
#        if row['genres'] != top :
#            filtered_df.drop(index, inplace=True)
#filtered_df

In [ ]:
filtered_df.head()

In [ ]:
#top_5_genres = ['animation', 'mystery', 'adventure', 'sci-fi', 'comedy']
top_5_df = adventure.append(animation)
top_5_df = top_5_df.append(mystery)
top_5_df = top_5_df.append(adventure)
top_5_df = top_5_df.append(scifi)
top_5_df = top_5_df.append(comedy)
top_5_df['genres'].unique()

In [ ]:
top_5_df = top_5_df.append(mystery)
top_5_df

In [ ]:
top_5_df = top_5_df.append(scifi)
top_5_df

In [ ]:
top_5_df['genres'].unique()